In [31]:
import pandas as pd
import numpy as np
df = pd.read_csv("data/streaming_titles.csv")

In [39]:
# Get genres as a set

genrelistraw = df["listed_in"].to_list()

genres = set()
for elem in genrelistraw:
    if type(elem) != str:
        continue
    splitted = elem.split(', ')
    for k in splitted:
        genres.add(k)

print(sorted(list(genres)))
print(len(genres))

['Action', 'Action & Adventure', 'Action-Adventure', 'Adult Animation', 'Adventure', 'Animals & Nature', 'Animation', 'Anime', 'Anime Features', 'Anime Series', 'Anthology', 'Arthouse', 'Arts', 'Biographical', 'Black Stories', 'British TV Shows', 'Buddy', 'Cartoons', 'Children & Family Movies', 'Classic & Cult TV', 'Classic Movies', 'Classics', 'Comedies', 'Comedy', 'Coming of Age', 'Concert Film', 'Cooking & Food', 'Crime', 'Crime TV Shows', 'Cult Movies', 'Dance', 'Disaster', 'Documentaries', 'Documentary', 'Docuseries', 'Drama', 'Dramas', 'Entertainment', 'Faith & Spirituality', 'Faith and Spirituality', 'Family', 'Fantasy', 'Fitness', 'Game Show / Competition', 'Game Shows', 'Health & Wellness', 'Historical', 'History', 'Horror', 'Horror Movies', 'Independent Movies', 'International', 'International Movies', 'International TV Shows', 'Kids', "Kids' TV", 'Korean TV Shows', 'LGBTQ', 'LGBTQ Movies', 'LGBTQ+', 'Late Night', 'Latino', 'Lifestyle', 'Lifestyle & Culture', 'Medical', 'Mili

A quick look:
There appears to be multiple of the same genre, as in, 
- Action-Adventure, Action & Adventure 
- Documentaries, Documentary, Docuseries

There is also an issue because one genre is called "Arts, Entertainment, and Culture" which gets parsed as three different genres.

We should check to make sure no other genre names have commas in them like this.